In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [34]:
features_raw_df = pd.read_csv('./features_raw.csv')
emotions_df = pd.read_csv('./emotions.csv')



In [35]:
combined_df = features_raw_df.join(emotions_df[['label']])

X = df.drop('label', axis=1).values
y = df['label'].values

# Scale your features - important for neural network convergence
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split your data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

NameError: name 'df' is not defined

In [ ]:
# Handle NaN values in numeric columns
numeric_cols = combined_df.select_dtypes(include=[np.number])
for col in numeric_cols.columns:
    # Fill columns with all NaN values with a placeholder (e.g., 0) or consider dropping them
    if numeric_cols[col].isnull().all():
        combined_df[col] = combined_df[col].fillna(0)
    else:
        # Fill remaining NaNs with the column mean
        combined_df[col] = combined_df[col].fillna(numeric_cols[col].mean())

# For non-numeric columns, decide on a strategy
non_numeric_cols = combined_df.select_dtypes(exclude=[np.number])
combined_df[non_numeric_cols.columns] = non_numeric_cols.fillna('unknown')

# Check for any remaining NaN values and print the problematic columns
nan_columns = combined_df.columns[combined_df.isnull().any()].tolist()
if nan_columns:
    print(f"Columns with NaN: {nan_columns}")
    for col in nan_columns:
        print(f"NaN count in '{col}': {combined_df[col].isnull().sum()}")
else:
    print("No NaN values remain.")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# Define the model
model = Sequential([
    # RNN layer
    # Note: 'input_shape' should match the shape of your features. 
    # For simplicity, we're assuming each feature is treated as a separate timestep.
    # Adjust 'input_shape' as per your dataset's feature size.
    SimpleRNN(50, input_shape=(X_train.shape[1], 1), activation='relu'),
    
    # Output layer
    # The number of neurons in the last layer should match the number of classes.
    # Use 'softmax' activation for multi-class classification.
    Dense(y_encoded.max() + 1, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


In [ ]:
# Replace Inf values with NaN
X_train[np.isinf(X_train)] = np.nan

# Compute the mean of each column, ignoring NaNs
col_mean = np.nanmean(X_train, axis=0)

# Check if there are any NaN values in the column means (which would imply that those columns are all NaN)
nan_cols = np.where(np.isnan(col_mean))

# If there are, you might choose to set these all-NaN columns to zero or some other value that makes sense for your dataset
X_train[:, nan_cols] = 0

# Recalculate the mean of each column, ignoring NaNs, after replacing all-NaN columns
col_mean = np.nanmean(X_train, axis=0)

# Find indices of NaNs in X_train and replace them with the mean of their column
inds = np.where(np.isnan(X_train))
X_train[inds] = np.take(col_mean, inds[1])

# Verify there are no more NaNs or Infs
assert not np.isnan(X_train).any(), "NaNs remain in X_train after cleaning."
assert not np.isinf(X_train).any(), "Infs remain in X_train after cleaning."

# Now that the data is clean, reshape it for the RNN
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Model configuration
n_units = 50
dropout_rate = 0.2
optimizer_learning_rate = 0.001
batch_size = 32
n_epochs = 10
n_features = X_train_reshaped.shape[2]  # Assuming X_train_reshaped has the shape (n_samples, n_timesteps, n_features)
n_classes = y_encoded.max() + 1  # Assuming y_encoded contains encoded class labels starting from 0

# Define the RNN model
model = Sequential()
# Adding an LSTM layer with dropout; return_sequences=True is needed if you stack LSTM layers
model.add(LSTM(units=n_units, input_shape=(None, n_features), activation='tanh', return_sequences=True))
model.add(Dropout(rate=dropout_rate))
# If you want to add another LSTM layer, make sure return_sequences is True in the first LSTM layer
model.add(LSTM(units=n_units, activation='tanh'))
model.add(Dropout(rate=dropout_rate))
# Output layer with softmax activation for classification
model.add(Dense(units=n_classes, activation='softmax'))

# Compile the model with the Adam optimizer and learning rate with gradient clipping
optimizer = Adam(learning_rate=optimizer_learning_rate, clipvalue=1.0)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
history = model.fit(
    X_train_reshaped, 
    y_train, 
    batch_size=batch_size, 
    epochs=n_epochs, 
    validation_split=0.2
)
